### NRLB with Pytorch
https://www.pnas.org/content/115/16/E3692

In [ ]:
print('here...')

In [ ]:
import numpy as np

import torch
import torch.utils.data as tdata
import torch.nn as nn
import torch.nn.functional as tfunc
import torch.optim as topti

torch.cuda.is_available()

In [ ]:

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))


In [ ]:

# Class for reading training/testing dataset files.
class toyDataset(tdata.Dataset):
    def __init__(self, dataFile, labelFile):
        # Load data from files.
        self.inputs = np.loadtxt(dataFile, dtype = np.float32).reshape(-1, 4, 1000)
        self.labels = np.loadtxt(labelFile, dtype = np.float32)

        self.length = len(self.labels)

    def __getitem__(self, index):
        # Return a single input/label pair from the dataset.
        inputSample = self.inputs[index]
        labelSample = self.labels[index]
        sample = {"input": inputSample, "label": labelSample}

        return sample

    def __len__(self):

        return self.length


In [ ]:
import matplotlib.pyplot as plt
from os.path import join
from os.path import exists

Here we define the functions to generate one hot encoded dna sequences

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
class hot_dna:
    def __init__(self,fasta):
   
        #check for and grab sequence name
        if re.search(">",fasta):
            name = re.split("\n",fasta)[0]
            sequence = re.split("\n",fasta)[1]
        else:
            name = 'unknown_sequence'
            sequence = fasta

        #get sequence into an array
        seq_array = array(list(sequence))

        #integer encode the sequence
        label_encoder = LabelEncoder()
        integer_encoded_seq = label_encoder.fit_transform(seq_array)

        #one hot the sequence
        onehot_encoder = OneHotEncoder(sparse=False)
        #reshape because that's what OneHotEncoder likes
        integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
        onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)

        #add the attributes to self 
        self.name = name
        self.sequence = fasta
        self.integer = integer_encoded_seq
        self.onehot = onehot_encoded_seq

Similarity between DNA sequences

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

import itertools

def mismatch(word, letters, num_mismatches):
    for locs in itertools.combinations(range(len(word)), num_mismatches):
        this_word = [[char] for char in word]
        for loc in locs:
            orig_char = word[loc]
            this_word[loc] = [l for l in letters if l != orig_char]
        for poss in itertools.product(*this_word):
            yield ''.join(poss)

### Motif generation function. Not required.

In [ ]:

# given a motif generate a seq of sequences with this seuquence embedded
def get_x_and_y(motif, batch=100):
    import random
    random.seed(500)

    import numpy as np
    nseqs = 500
    length = 15
    fg_seqs = np.array([scr.gen.get_random_sequence(length) + 'ACGT' for k in range(nseqs)])
    # bg_seqs = np.array([scr.gen.get_random_sequence(length) for k in range(nseqs)])
    # bg_seqs = [scr.gen.randomize_sequence(fg_seqs[i]) + 'ACGT' for i in range(len(fg_seqs))]

    options = []
    for n_mismatches in range(0, min(len(motif), 5)):
        next_options = np.random.choice(list(mismatch(motif, 'ACGT', n_mismatches)), 100)[:100]
        print(n_mismatches, len(next_options), next_options[:10])
        options += list(next_options)
    y = []
    for i, opt in zip(range(len(fg_seqs)), options):
        p = np.random.choice(range(len(fg_seqs[0]) - 4 - len(opt)))
        fg_seqs[i] = fg_seqs[i][:p] + opt + fg_seqs[i][p + len(opt):]
        y.append(int(similar(motif, opt) * batch))
    y_pos = np.array(y)
    fg_seqs = fg_seqs[:len(y_pos)]
    print(len(fg_seqs))
    return fg_seqs, y_pos

### Read HT-SELEX data

In [ ]:
import pandas as pd

In [ ]:
nrlb_data = 'https://www.dropbox.com/s/oib5lq23wck3gsh/GATA3_TGTCGT20NGA_AC_4.tsv.gz'
!wget $nrlb_data -O GATA3_TGTCGT20NGA_AC_4.tsv.gz

In [10]:
import gzip
def get_seqs(fastq_path, round_key):
    fastq = np.array([(s.strip()).decode('utf-8') for s in gzip.open(fastq_path)])
    mask = np.array([((i + 3) % 4 == 0) for i in range(len(fastq))])
    seqs = fastq[mask]
    df = pd.DataFrame(seqs, columns=['seq'])
    df = df[~df['seq'].str.contains('N')]
    seq_counts = df['seq'].value_counts()
    df['counts'] = [seq_counts.loc[s] for s in df['seq']]
    df['round'] = round_key
    df = df.drop_duplicates('seq').sort_values('counts', ascending=False)
    return(df)

In [12]:
# download CTCF from motif central
!wget http://pbdemo.x3dna.org/files/example_data/singleTF/countTable.0.CTCF_r3.tsv.gz -O data/countTable.0.CTCF_r3.tsv.gz

data/countTable.0.CTCF_r3.tsv.gz: No such file or directory


In [13]:
data = pd.read_csv('data/countTable.0.CTCF_r3.tsv.gz', sep='\t', header=None) # ['sequence', 'round.0', 'round.1']) #  header=False)
data.columns = ['seq', 0, 1]

FileNotFoundError: [Errno 2] No such file or directory: 'data/countTable.0.CTCF_r3.tsv.gz'

In [ ]:
data

In [ ]:
# # r0 = get_seqs('MAX_R0.fastq.gz', 0)
# # r1 = get_seqs('MAX_R1.fastq.gz', 1)
# r0 = get_seqs('data/ZeroCycle_ES0_TGTCGT20NGA_0.txt.gz', 0)
# r1 = get_seqs('data/GATA3_TGTCGT20NGA_AC_1.fastq.gz', 1)

In [ ]:
# data']

In [ ]:
# r1.sort_values('counts')

In [ ]:
# r0.shape, r1.shape

In [ ]:
# r0 = r0.head(10000)
# r1 = r1.head(10000)
# # r1['seq'] = r1['seq'].str[:5] + 'GATA' + r1['seq'].str[9:]

In [ ]:
r1 = data.head(10000)

In [ ]:
control = 0
if control:
    seq_len = 9
    r0 = []
    for seq in itertools.product('ATCG', repeat=seq_len):
        seq = ''.join(seq)
        r0.append([seq, 1, 0])
    r0 = pd.DataFrame(r0, columns=['seq', 'counts', 'round'])
    r0['prob'] = r0['counts'] / sum(r0['counts'])
    r0 = r0.sample(1000)

    r1 = []
    for seq in itertools.product('ATCG', repeat=seq_len):
        seq = ''.join(seq)
        if np.random.random() > .3:
            seq = seq[:3] + 'GATA' + seq[7:]
        if np.random.random() > .6:
            seq = seq[:3] + 'CATA' + seq[7:]

        r1.append([seq, 20 if 'GATA' in seq else 10 if 'CATA' in seq else 5, 0])
    r1 = pd.DataFrame(r1, columns=['seq', 'counts', 'round'])
    r1['prob'] = r1['counts'] / sum(r1['counts'])
    r1 = r1.sample(1000)


In [ ]:
def write_fastq_from_seqs_table(seqs, path):
    # write fastq.gz from seqs
    seqs = [r['seq'] for ri, r in seqs.iterrows() for i in range(r['counts'])]
    writer = gzip.open(path, 'wt')
    for s in seqs:
        writer.write('@\n')
        writer.write(s + '\n')
        writer.write('+\n')
        writer.write('@\n')
    writer.close()


In [ ]:
# write_fastq_from_seqs_table(r1, 'r1.fastq.gz')
# write_fastq_from_seqs_table(r0, 'r0.fastq.gz')

In [ ]:
# paste a GATA motif
# r1['seq'] = np.where(r1['counts'] == 2, r1['seq'].str[:5] + 'GATA' + r1['seq'].str[9:], r1['seq'])
# r1['seq'] = np.where(r1['counts'] == 2, 'AAAAAAAAAAAAAAAA', 'GGGGGGGGGGGGGGGG') # r1['seq'])

In [ ]:
data

In [ ]:
kmers_table = {} # round # k
kstart = 13
kstop = kstart + 1

pseudocount = 1
for selection_round, df in zip([0, 1], [r1, r1]):
    kmer_table_by_k = {}
    for k in range(kstart, kstop): # this is also the kernel size
        print('selection round', selection_round)
        print('generating a k-mers table for k=%i' % k)
        unique_kmers = [[r['seq'][si: si + k], r[selection_round] + 1]  for ri, r in df.iterrows() for si in range(len(r['seq']) - k + 1)]
        # kmer_table = pd.Series(unique_kmers).value_counts()
        # kmer_prob = kmer_table / kmer_table.sum()
        kmer_table = pd.DataFrame(unique_kmers, columns=['kmer', 'counts'])
        kmer_table = kmer_table.groupby('kmer').sum().sort_values('counts', ascending=False)
        kmer_table['prob'] = kmer_table['counts'] / kmer_table['counts'].sum()
        kmer_table['round'] = selection_round
        kmer_table_by_k[k] = kmer_table
    kmers_table[selection_round] = kmer_table_by_k

In [ ]:
ksel = kstart
kmers = pd.concat([kmers_table[sel_round][ksel] for sel_round in kmers_table])
kmers
kmers.sort_values('prob', ascending=False)

In [ ]:
r1.sort_values(0, ascending=False)

In [ ]:
import seaborn as sns
sns.displot(np.log(r1[0] + 1))
plt.xlabel('counts [log]')

In [ ]:
r1[0].value_counts()

In [ ]:
# calc_Z(kmers_r0, 4)

In [ ]:
def get_wi(seq, kmers_r0, k):
    wi = 0
    for si in range(0, len(seq) - k + 1):
        kmer = seq[si: si + k]
        wi += sum([kmers_r0.loc[kmer]['prob']]) if kmer in kmers_r0.index else 0
    return wi

In [ ]:
kmers_r1 = kmers_table[1][ksel]
kmers_r0 = kmers_table[0][ksel]

print('preparing p_i0 for r0')
w = []
for ri, r in r1.iterrows():
    if len(w) % 1000 == 0:
        print(len(w), 'out of', r1.shape[0])
    seq = r['seq']
    # print(seq)
    wi = get_wi(seq, kmers_r1, ksel)
    # print(seq, wi)
    w.append(wi)
    # break    
r1['p_i0'] = w


print('preparing p_i0 for r1')
# do the same for r1
w = []
for ri, r in r1.iterrows():
    if len(w) % 1000 == 0:
        print(len(w), 'out of', r1.shape[0])
    seq = r['seq']
    # print(seq)
    wi = get_wi(seq, kmers_r0, ksel)
    # print(seq, wi)
    w.append(wi)
    # break    
r1['p_i1'] = w



In [ ]:
# Z0 is a constant so arguably we do not need it
def calc_Z(kmers_table, seq_len):
    for seq in itertools.product('ATCG', repeat=seq_len):
        print(''.join(seq))
    return

In [ ]:
### define models

# Class for creating the neural network.
class nrlb(nn.Module):
    def __init__(self, n_bp, kernel_sizes=[10], init_scale=1e-3, initial_conv1d=None,
                random_init=0):
        super(nrlb, self).__init__()

        # Create and initialise weights and biases for the layers.
        self.conv1d_set = nn.ModuleList()    
        for k in kernel_sizes:
            conv1d = nn.Conv1d(4, 1, k)
            if initial_conv1d is not None:
                print(conv1d.weight.data)
                conv1d.weight.data = initial_conv1d
                print(conv1d.weight.data)
            else:
                if not random_init:
                    print('zetting weights to zero')
                    conv1d.weight.data.fill_(0.0)
            self.conv1d_set.append(conv1d)
        # self.scale_set = [nn.Parameter(torch.FloatTensor([init_scale])) for k in kernel_sizes]
  
        # self.batchnorm = torch.nn.BatchNorm1d(1)

        # self.softmax_out = nn.Softmax(1)
        self.linear1 = nn.Linear(1, 1)
#         self.linear2 = nn.Linear(1, 1)
#         self.linear3 = nn.Linear(1, 1)
#         self.linear4 = nn.Linear(1, 1)
#         self.linear5 = nn.Linear(1, 1)

        # self.fc2 = nn.Linear(n_bp - kernel_size + 1, 1)

    def forward(self, x1_fwd, x1_rev, x2_fwd, x2_rev):
        # Create the forward pass through the network.
        
        res = None # np.zeros(x1.shape[0])
        for i, conv1d in enumerate(self.conv1d_set):
            
            out1 = conv1d(x1_fwd)
            out2 = conv1d(x1_rev)

            out1 = out1.squeeze()
            out2 = out2.squeeze()
            
            out1 = out1 * x2_fwd
            out2 = out2 * x2_rev

            # print(out1.shape)
            # print(sum(out))
            out = torch.sum(out1, axis=1) + torch.sum(out2, axis=1)
            
            # out = out * self.scale_set[i]

            if res is None:
                res = out
            else:
                res += out

        # print(res.unsqueeze(-1))
        # print(res.shape)
        # res = self.softmax_out(res.unsqueeze(-1))

        # res = torch.log(res + 1e-15)
        # res = self.batchnorm(res.unsqueeze(-1))

        res = self.linear1(res.unsqueeze(-1))
        # res = self.linear2(res)
        # res = self.linear3(res)
        # res = self.linear4(res)
        # res = self.linear5(res)

        # print(res.shape)

        return res


In [ ]:
r1[[1, 0]].sum()

In [ ]:
# ## x_test = torch.rand(1).to(device='cuda')
# x_test = torch.rand([5, 4, 20]).to(device='cuda')
# # print(x_test)
# out_test = torch.sum(model.conv1d_set[0](x_test), axis=2)
# # print(model.softmax_out(out_test))
# # print(model.linear_out(out_test))

# model.linear3(out_test)

In [ ]:
# # r1_sel['probe.enrichment'] = r1_sel['p_i1'] / r1_sel['p_i0']
# # r1_sel = r1_sel.sort_values('probe.enrichment', ascending=False)
# r1_sel[r1_sel['seq'].str.contains('GATA') | r1_sel['seq'].str.contains('TATC')]

In [ ]:
print('\n######\nQuery %i' % k)
print('preparing data...')
x = np.array(r1['seq'])

r1['p_i1'] = r1[1] / sum(r1[1])
# plt.scatter(r1['p_i0'], r1['p_i1'])

y = np.array(r1['p_i1'], dtype=np.float32)


print(r1.shape)

In [ ]:
r1 = r1.sort_values(1, ascending=False)
r1['ratio_1_over_0'] = np.log((r1['p_i1'] + 1) / (r1['p_i0'] + 1))
r1.shape

In [ ]:
r1.sort_values('ratio_1_over_0')

In [ ]:
plt.hist(r1['ratio_1_over_0'])

In [ ]:
y = np.array(r1['ratio_1_over_0'], dtype=np.float32)
# y = np.array(r1_sel['probe.enrichment'], dtype=np.float32)
# y = np.array(r1['counts'], dtype=np.float32)
# r1['ratio_1_over_0']

In [ ]:
from Bio.Seq import Seq
# str(seq.reverse_complement())

print('preparing one encoding representations...')
# create the one hot encoding representations. This needs to be appended by the dictionary size to avoid smaller matrices when some nucleotides not found.
x1_fwd = np.array([hot_dna(x[i] + 'ACGT').onehot.T[:,:-4].astype(np.float32) for i in range(len(x))])
x1_rev = np.array([hot_dna(str(Seq(x[i]).reverse_complement()) + 'ACGT').onehot.T[:,:-4].astype(np.float32) for i in range(len(x))])
print('done...')



pmin = min(kmers_table[0][ksel]['prob'])

x2_fwd = np.array([np.array([kmers_table[0][ksel].loc[xi[si: si + k]]['prob'] if xi[si: si + k] in kmers_table[0][ksel] else pmin
                             for si in range(len(xi) - k + 1)], dtype=np.float32)
                   for xi in x], dtype=np.float32)

x2_rev = np.array([np.array([kmers_table[0][ksel].loc[str(Seq(xi[si: si + k]).reverse_complement())]['prob']
                             if str(Seq(xi[si: si + k]).reverse_complement()) in kmers_table[0][ksel] else pmin
                             for si in range(len(xi) - k + 1)], dtype=np.float32)
                   for xi in x], dtype=np.float32)


x2_fwd = x2_fwd.reshape(x2_fwd.shape[:2])
print(x2_fwd.shape)

x2_rev = x2_rev.reshape(x2_rev.shape[:2])
print(x2_rev.shape)

scale_01 = False
if scale_01:
    y = (y - y.min()) / (y.max() - y.min())
    
# std norm 
y = (y - y.mean()) / y.std()


# x_train -= np.mean(x_train)
y += np.abs(np.min(y))

# y *= 1e-10

# tensors
x1_fwd_tensor = torch.FloatTensor(x1_fwd)
x1_rev_tensor = torch.FloatTensor(x1_rev)

x2_fwd_tensor = torch.FloatTensor(x2_fwd)
x2_rev_tensor = torch.FloatTensor(x2_rev)

y_tensor = torch.FloatTensor(y)

n_bp = len(x[0])
print(n_bp)


In [ ]:
len(list(r1['seq'])[0])

In [ ]:
plt.hist(y)

In [ ]:
y_tensor

In [ ]:
# out = conv1d(xi)
# out_df = pd.DataFrame(out.squeeze().detach().numpy())
# # print(out_df.sum(axis=1))
# torch.sum(out.squeeze(), axis=1) * p_i0
# # out = out.squeeze()

In [ ]:
# x2_tensor

In [ ]:
import seaborn as sns

In [ ]:
k

In [ ]:
n_epochs = 10000
print('# of base pairs', n_bp)
print('kernel size (also k-mer length)', k)
init_scale = 1

K = torch.Tensor([[[0, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]]])

K = torch.Tensor([[[0] * k,
                   [0] * k,
                   [0] * k,
                   [0] * k]])

model = nrlb(n_bp, kernel_sizes=[k], init_scale=init_scale, # initial_conv1d=K,
            random_init=1).to(device) # Create an instance of the network in memory (potentially GPU memory).
# net = netsimple(n_bp, kernel_sizes=[k], init_scale=init_scale).to(device) # Create an instance of the network in memory (potentially GPU memory).
criterion = nn.MSELoss() # Add a sigmoid activation function to the output.  Use a binary cross entropy
                                    # loss function.
optimiser = topti.Adam(model.parameters(), lr = 0.001) # Minimise the loss using the Adam algorithm.
# optimiser = topti.LBFGS(net.parameters(), lr = 0.01) # Minimise the loss using the Adam algorithm.


ppms = []
for ppm in model.conv1d_set:
    ppm = ppm.weight.data.cpu().numpy()
    ppm = pd.DataFrame(ppm[0])    
    ppms.append(ppm)
for i, ppm in enumerate(ppms):
    plt.subplot(len(ppms), 1, i + 1)
    sns.heatmap(ppm, cmap='Reds')
plt.show()

log_each = int(n_epochs / 10)
## TRAIN
print('Training...')
# print('shapes', x1_tensor.shape, x2_tensor.shape, y.shape)

labels = y_tensor

use_gpu = True
if use_gpu:
    x1_fwd_tensor = x1_fwd_tensor.to(device='cuda')
    x1_rev_tensor = x1_rev_tensor.to(device='cuda')
    x2_fwd_tensor = x2_fwd_tensor.to(device='cuda')
    x2_rev_tensor = x2_rev_tensor.to(device='cuda')
    labels = labels.to(device='cuda')


In [ ]:
x2_fwd_tensor.shape

In [ ]:
model.conv1d_set[0].weight

In [ ]:
x1_fwd_tensor.shape, x1_rev_tensor.shape, x2_fwd_tensor.shape, x2_rev_tensor.shape

In [ ]:
model

In [ ]:
for epoch in range(n_epochs):

    optimiser.zero_grad()

    # print(inputs.shape, labels.shape)
    outputs = model(x1_fwd_tensor, x1_rev_tensor, x2_fwd_tensor, x2_rev_tensor) # Forward pass through the network.
    # outputs = net(inputs) # Forward pass through the network.

    # loss = -torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))

    loss = criterion(outputs, labels)
    # def f():
    #     return loss
    # optimiser.step(f) # Step to minimise the loss according to the gradient.

    loss.backward() # Calculate gradients.
    optimiser.step() # Step to minimise the loss according to the gradient.

    if epoch % log_each == 0:
        print("Epoch: %2d, Loss: %.12f" % (epoch + 1, loss))

In [ ]:
from sklearn import metrics
import logomaker

In [ ]:
r1.sort_values('ratio_1_over_0', ascending=False)

In [ ]:
y

In [ ]:
x2_fwd_tensor[1]

In [ ]:
# calculate predictions
ytrue = []
ypred = []
with torch.no_grad():
    # Get a batch and potentially send it to GPU memory.
    # inputs, labels = batch["input"].to(device), batch["label"].to(device)
    # inputs = x1_tensor
    labels = y_tensor

    # print(inputs.shape)
    # outputs = torch.sigmoid(net(x1_tensor, x2_tensor))
    outputs = model(x1_fwd_tensor, x1_rev_tensor, x2_fwd_tensor, x2_rev_tensor).cpu() # Forward pass through the network.
    # predicted = torch.round(outputs)
    predicted = outputs

    ytrue += list(labels)
    ypred += list(outputs)

print('true', ytrue[:5])
print('pred', ypred[:5])

# print('MSE default', metrics.mean_squared_error(y, .sum(axis=1)))
print('MSE final', metrics.mean_squared_error(ytrue, ypred))

print(len(ytrue), len(ypred))

## downstream plots: compare before and after optimization effects
# prior relationship between kmer counts and observed 
# plt.subplot(1, 2, 1)

# ypred_init = np.array(x2)
# ypred_init = np.array(x2.sum(axis=1))

# plt.scatter(np.array(ytrue), ypred_init, s=10, c='red')
# plt.xlabel('counts [norm]')
# plt.ylabel('sum of k-mer prob')

plt.subplot(1, 2, 2)
plt.scatter(np.array(ytrue), np.array(ypred), s=10)
plt.xlabel('observed value')
plt.ylabel('predicted value')

plt.tight_layout()
plt.show()

ppms = []
for ppm in model.conv1d_set:
    ppm = ppm.weight.data.cpu().numpy()
    print(ppm)
    ppm = pd.DataFrame(ppm[0])
    ppm.index = 'A', 'C', 'G', 'T'
    ppms.append(ppm)
for i, ppm in enumerate(ppms):
    plt.subplot(len(ppms), 1, i + 1)
    
    cmax, cmin = max(ppm.max()), min(ppm.min())
    
    sns.heatmap(ppm, cmap='Reds', vmin=cmin, vmax=cmax)
    print(ppm)
    # create Logo object
    crp_logo = logomaker.Logo(ppm.T,
                          shade_below=.5,
                          fade_below=.5,
                          font_name='Arial Rounded MT Bold')
    plt.show()

    
# plt.show()

# print(model.scale_set)
# print(net, ytrue, ypred)
ppm

In [ ]:
max(ypred), max(ytrue), min(ypred), min(ytrue)

## Visualization

In [ ]:
print('here...')

In [ ]:
np.array(ypred)

In [ ]:
plt.scatter(np.array(ytrue), np.array(ypred))
# plt.autoscale(enable=False, axis='y')

In [ ]:
plt.autoscale?

In [ ]:
plt.hist(np.array(ypred.cpu()))

In [ ]:
# !pip install torchviz

In [ ]:
model

In [ ]:
from torchviz import make_dot

make_dot(outputs.mean(), params=dict(list(model.named_parameters()))).render("attached") # rnn_torchviz", format="png")
# make_dot(r).render("attached", format="png")

In [ ]:
!readlink -f .

In [ ]:
for layer in model.conv1d_set:
    print(layer.weight.data)
model.linear1.bias, model.linear1.weight
# print(layer.weight.data)

In [ ]:
# # fastq = np.array([(s.strip()).decode('utf-8') for s in gzip.open(nrlb_data)])
# # print(len(fastq))
# # mask = np.array([((i + 3) % 4 == 0) for i in range(len(fastq))])
# # seqs = fastq[mask] 
# df = pd.read_csv('GATA3_TGTCGT20NGA_AC_4.tsv.gz', index_col=0)
# # df['seq'] += df['seq'] + ''
# # df = df[~df['seq'].str.contains('N')]
# # seq_counts = df['seq'].value_counts()
# # df['counts'] = [seq_counts.loc[s] for s in df['seq']]
# # df = df.drop_duplicates('seq').sort_values('counts', ascending=False)

# # selex_path = '/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/screg/notebooks/protein_dna_landscape/ELK1_EVX1_3_AAD_TGGAAT40NAAT.fastq.tsv'
# # df = pd.read_csv(selex_path, sep='\t', header=None)
# # df.columns = ['kmer', 'counts']

# print(df.shape)

In [ ]:

# # Class for reading training/testing dataset files.
# class toyDataset(tdata.Dataset):
#     def __init__(self, dataFile=None, labelFile=None, data_x=None, data_y=None):
#         # Load data from files.
#         self.inputs = [np.loadtxt(dataFile, dtype = np.float32).reshape(-1, 4, 1000) if dataFile is not None else data_x[0],
#                        np.loadtxt(dataFile, dtype = np.float32).reshape(-1, 1, 1000) if dataFile is not None else data_x[1]]
        
#         self.labels = np.loadtxt(labelFile, dtype = np.float32) if dataFile is not None else data_y

#         self.length = len(self.labels)

#     def __getitem__(self, index):
#         # Return a single input/label pair from the dataset.
#         inputSample = self.inputs[index]
#         labelSample = self.labels[index]
#         sample = {"input": inputSample, "label": labelSample}

#         return sample

#     def __len__(self):

#         return self.length


The counts distribution for the observed data (log)

## Initial parms for ppms

In [ ]:
import sklearn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
def get_trained_model(df, kmer_table_by_k, k, n_epochs=10, nsel=2000, log_each=250, init_scale=10000):
    
    print('\n######\nQuery %i' % k)
    print('preparing data...')
    x = np.array(df['seq'].head(nsel))
    y = np.array(df['counts'].head(nsel), dtype=np.float32)
    # create the one hot encoding representations. This needs to be appended by the dictionary size to avoid smaller matrices when some nucleotides not found.
    x_one_hot = np.array([hot_dna(x[i] + 'ACGT').onehot.T[:,:-4].astype(np.float32) for i in range(len(x))])
    x1 = x_one_hot
    x2 = np.array([np.array([kmer_table_by_k[k].loc[xi[si: si + k]]['prob'] for si in range(len(xi) - k + 1)], dtype=np.float32)
                   for xi in x], dtype=np.float32)
    x2 = x2.reshape(x2.shape[:2])
    y = (y - y.min()) / (y.max() - y.min())
    
    # tensors
    x1_tensor = torch.FloatTensor(x1)
    x2_tensor = torch.FloatTensor(x2)
    y_tensor = torch.FloatTensor(y)
    
    n_bp = len(x[0])
    print(n_bp)

    
    print('# of base pairs', n_bp)
    print('kernel size (also k-mer length)', k)
    net = netsimple(n_bp, kernel_sizes=[k], init_scale=init_scale).to(device) # Create an instance of the network in memory (potentially GPU memory).
    criterion = nn.MSELoss() # Add a sigmoid activation function to the output.  Use a binary cross entropy
                                        # loss function.
    optimiser = topti.Adam(net.parameters(), lr = 0.01) # Minimise the loss using the Adam algorithm.
    
    ppms = []
    for ppm in net.conv1d_set:
        ppm = ppm.weight.data.numpy()
        ppm = pd.DataFrame(ppm[0])    
        ppms.append(ppm)
    for i, ppm in enumerate(ppms):
        plt.subplot(len(ppms), 1, i + 1)
        sns.heatmap(ppm, cmap='Reds')

    ## TRAIN
    print('Training...')
    print('shapes', x1_tensor.shape, x2_tensor.shape, y.shape)
    for epoch in range(n_epochs):
        inputs, labels = x1_tensor, y_tensor

        optimiser.zero_grad()

        # print(inputs.shape, labels.shape)
        outputs = net(x1_tensor, x2_tensor) # Forward pass through the network.
        # outputs = net(inputs) # Forward pass through the network.

        # loss = -torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))

        loss = criterion(outputs, labels)

        loss.backward() # Calculate gradients.
        optimiser.step() # Step to minimise the loss according to the gradient.

        if epoch % log_each == 0:
            print("Epoch: %2d, Loss: %.3f" % (epoch + 1, loss))
         
    # calculate predictions
    ytrue = []
    ypred = []
    with torch.no_grad():
        # Get a batch and potentially send it to GPU memory.
        # inputs, labels = batch["input"].to(device), batch["label"].to(device)
        inputs, labels = x1_tensor, y_tensor

        # print(inputs.shape)
        # outputs = torch.sigmoid(net(x1_tensor, x2_tensor))
        outputs = net(x1_tensor, x2_tensor)
        # predicted = torch.round(outputs)
        predicted = outputs

        ytrue += list(labels)
        ypred += list(outputs)

    print('MSE default', metrics.mean_squared_error(y, x2.sum(axis=1)))
    print('MSE final', metrics.mean_squared_error(ytrue, ypred))
    
    print(len(ytrue), len(ypred))

    ## downstream plots: compare before and after optimization effects
    # prior relationship between kmer counts and observed 
    plt.subplot(1, 2, 1)
    
    ypred_init = np.array(x2.sum(axis=1))
    
    plt.scatter(np.array(ytrue), ypred_init, s=10, c='red')
    plt.xlabel('counts [norm]')
    plt.ylabel('sum of k-mer prob')

    plt.subplot(1, 2, 2)
    plt.scatter(np.array(ytrue), np.array(ypred), s=10)
    plt.xlabel('observed')
    plt.ylabel('predicted')

    plt.tight_layout()
    plt.show()
    
    return net, ytrue, ypred

In [ ]:
for ki in range(kstart, kstop):
    
    # this is a function call to the whole pipeline routine.
    nsel = 10000 # df.shape[0] # 10000
    net, ytrue, ypred = get_trained_model(df, kmer_table_by_k, ki, n_epochs=2000, log_each=250, nsel=nsel)
    
    ppms = []
    for ppm in net.conv1d_set:
        ppm = ppm.weight.data.numpy()
        ppm = pd.DataFrame(ppm[0])    
        ppms.append(ppm)
    for i, ppm in enumerate(ppms):
        plt.subplot(len(ppms), 1, i + 1)
        sns.heatmap(ppm, cmap='Reds')

    plt.show()
    
    print(net.scale_set)
    
    ### Visualize results (not working without screg)
    plot_logo = False
    if plot_logo:
        import screg as scr
        scr.constants.ANNOTATIONS_DIRECTORY = '/mnt/znas/icb_zstore01/groups/ml01/datasets/annotations'
        scr.constants.SCREG_DATA_DIRECTORY = '/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/screg/data'

        for mi, motif in enumerate(ppms):
            ppm = pd.DataFrame(motif)
            norm = True
            if norm:
                for c in ppm:
                    # print(c)
                    ppm[c] = (ppm[c] - min(ppm[c])) / (max(ppm[c]) - min(ppm[c]))
                    ppm[c] = ppm[c] / sum(ppm[c])
            ppm.index = 'A', 'C', 'G', 'T'
            print(mi * 2 + 1)
            ax = plt.subplot(len(ppms), 2, mi * 2 + 1)
            sns.heatmap(ppm, cmap='Reds',)
            print(mi * 2 + 2)
            ax = plt.subplot(len(ppms), 2, mi * 2 + 2)
            
            scr.pl.plot_pwm_model('learned motif %i' % (mi + 1), ppm=ppm, ax=ax)
            plt.ylim([0, 1.3])
            plt.ylabel('Bits')
            ppm
            plt.tight_layout()
        plt.show()

### end. 

### Checking log conversions here

In [ ]:
import numpy as np
# create dummy data for training
x_values = [i for i in range(11)]

# log conversion
x_train = np.array(x_values, dtype=np.float32)
x_values = np.power(x_train, 10)
x_train = np.log10(x_values + 1)
x_train = x_train.reshape(-1, 1)

y_values = [2*i + 1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = np.log10(y_train + 1)

x_train += 100

x_train -= np.mean(x_train)
x_train += np.abs(np.min(x_train))

y_train = y_train.reshape(-1, 1)

In [ ]:
plt.scatter(x_train, y_train)

In [ ]:
import torch
from torch.autograd import Variable
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out


In [ ]:
inputDim = 1        # takes variable 'x' 
outputDim = 1       # takes variable 'y'
learningRate = 0.001
epochs = 10000

model = linearRegression(inputDim, outputDim)
##### For GPU #######
if torch.cuda.is_available():
    model.cuda()


In [ ]:
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)
for epoch in range(epochs):
    # Converting inputs and labels to Variable
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(x_train).cuda())
        labels = Variable(torch.from_numpy(y_train).cuda())
    else:
        inputs = Variable(torch.from_numpy(x_train))
        labels = Variable(torch.from_numpy(y_train))

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs)

    # get loss for the predicted output
    loss = criterion(outputs, labels)
    # print(loss)
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    if epoch % int(epochs / 10) == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

In [ ]:
x_train, y_train

In [ ]:
model.linear.weight.data.cpu().numpy(), model.linear.bias.data.cpu().numpy()[0]

In [ ]:
with torch.no_grad(): # we don't need gradients in the testing phase
    if torch.cuda.is_available():
        predicted = model(Variable(torch.from_numpy(x_train).cuda())).cpu().data.numpy()
    else:
        predicted = model(Variable(torch.from_numpy(x_train))).data.numpy()
    print(predicted)

plt.clf()
plt.plot(x_train, y_train, 'go', label='True data', alpha=0.5)
plt.plot(x_train, predicted, '--', label='Predictions', alpha=0.5)
plt.legend(loc='best')
plt.show()